# Cartographie de la Température Moyenne des Postes Météo en France


Le présent notebook est consacré à l'analyse et à la visualisation des données météorologiques issues de divers postes de mesure en France. Les données utilisées proviennent du fichier `Q_64_previous-1950-2023_RR-T-Vent.csv.gz`, disponible à l'adresse https://object.files.data.gouv.fr/meteofrance/data/synchro_ftp/BASE/QUOT/Q_64_previous-1950-2023_RR-T-Vent.csv.gz. Ce fichier contient des informations météorologiques recueillies à partir de divers postes de mesure, notamment la température, les précipitations, et la vitesse du vent.


L'objectif de cette analyse est de préparer les données pour une carte représentant les postes de mesure avec leur température moyenne respective. Cette visualisation permettra d'appréhender la répartition géographique des températures moyennes en France.


## Méthodologie

La méthodologie employée pour cette analyse comporte plusieurs étapes clés. Tout d'abord, les données sont chargées et nettoyées. Ensuite, une requête SQL est exécutée via DuckDB pour agréger les données par poste de mesure et calculer la moyenne de la température moyenne (`TM`) pour chaque année. Les résultats sont ensuite utilisés pour créer une visualisation interactive sous forme de carte utilisant la bibliothèque Folium. Les postes de mesure sont représentés par des cercles dont la couleur est fonction de la température moyenne enregistrée.


Les données sont ainsi analysées et représentées de manière à offrir une vue d'ensemble claire et intuitive de la répartition des températures moyennes en France. Cette visualisation facilite l'identification des tendances et des disparités régionales en matière de température.

## 🔧 Configuration

In [1]:
# Installation et imports
import duckdb as ddb
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

## 🦆 Chargement du dataset avec Duckdb

In [1]:
# Fonction de chargement complète (basée sur load_file_from_url_lite)
def load_file_from_url_lite(url_dataset="", loader="read_csv_auto", options="", nom_table="loaded_dataset", safe_mode=False):
    ddb.execute("install spatial")
    ddb.execute("load spatial")
    ddb.execute("INSTALL h3 FROM community")
    ddb.execute("LOAD h3")
    ddb.execute("set force_download=True")
    ddb.execute(f"drop table if exists {nom_table}")   
    
    # Détection automatique du type de fichier
    if 'csv' in url_dataset: 
        loader = "read_csv_auto"
    elif 'tsv' in url_dataset: 
        loader = "read_csv_auto"
    elif 'txt' in url_dataset: 
        loader = "read_csv_auto"
    elif 'parquet' in url_dataset: 
        loader = "read_parquet"
    elif 'json' in url_dataset: 
        loader = "read_json_auto"
    elif 'xls' in url_dataset or 'xlsx' in url_dataset: 
        loader = "st_read"
    elif 'shp' in url_dataset: 
        loader = "st_read"
    elif 'geojson' in url_dataset: 
        loader = "st_read"
    else: 
        raise ValueError(f"Type de fichier non supporté pour {url_dataset}")
    
    if options=="": 
        options = "" 
    if 'csv' in url_dataset and safe_mode==True: 
        options = ", all_varchar=1" 
    if nom_table=="": 
        nom_table = "loaded_dataset"
    
    try:
        status = ddb.sql(f"""
            create or replace table {nom_table} as select *
            from
            {loader}("{url_dataset}" {options})
        """)
        return status
    except Exception as e:
        return f"Erreur au chargement du fichier : {str(e)}"

def run_query(sql):
    return ddb.sql(sql.replace("`"," ")).to_df()

# Chargement des données
load_file_from_url_lite("https://object.files.data.gouv.fr/meteofrance/data/synchro_ftp/BASE/QUOT/Q_64_previous-1950-2023_RR-T-Vent.csv.gz", safe_mode=True)
print("✅ Données chargées avec succès")

## 🔍 Analyse SQL

Cette requête utilise des techniques SQL pour extraire et transformer les données de manière efficace.

In [2]:
# Exécution de la requête
df = run_query(""" SELECT 
  CAST(SUBSTRING("AAAAMMJJ", 1, 4) AS INTEGER) AS year,
  "NUM_POSTE",
  CAST("LON" AS DOUBLE) AS longitude,
  CAST("LAT" AS DOUBLE) AS latitude,
  AVG(CAST("TM" AS DOUBLE)) AS avg_TM
FROM 
  loaded_dataset
WHERE 
  "TM" IS NOT NULL
GROUP BY 
  "NUM_POSTE", "LON", "LAT", CAST(SUBSTRING("AAAAMMJJ", 1, 4) AS INTEGER)
ORDER BY 
  year """)
print(f"Résultats : {len(df)} lignes")
df.head()

## 📈 Visualisation

La bibliothèque principale utilisée est Folium, qui est une surcouche de Leaflet.js permettant de créer des cartes interactives. Cette technologie est adaptée pour représenter des données géolocalisées avec des marqueurs et des informations associées, comme c'est le cas ici avec les températures moyennes par poste. Folium permet de créer une visualisation interactive et intuitive.

In [3]:
import pandas as pd
import duckdb as ddb
import folium
import branca.colormap as cm

color_scale = cm.LinearColormap(colors=['dodgerblue', 'orangered'],
                                 index=[df['avg_TM'].min(), df['avg_TM'].max()],
                                 vmin=df['avg_TM'].min(), vmax=df['avg_TM'].max(),
                                 caption='Température moyenne (°C)')

# ordre croissant pour superposer les plus froid en-dessous
coords = df[['latitude', 'longitude', 'NUM_POSTE', 'avg_TM']].sort_values('avg_TM')

m = folium.Map(location=[coords.latitude.mean(), coords.longitude.mean()],
               tiles='CartoDB positron', zoom_start=8)

for _, row in coords.iterrows():
    folium.CircleMarker(
        location=[row.latitude, row.longitude],
        radius=6,
        color='black',
        weight=0.5,
        fillColor=color_scale(row.avg_TM),
        fillOpacity=0.85,
        popup=None,
        tooltip=f"<b>Poste&nbsp;n°{int(row.NUM_POSTE)}</b><br>{row.avg_TM:.2f}°C"
    ).add_to(m)

color_scale.add_to(m)

dataviz = m
dataviz.show()

---
*Made with ❤️ and with [duckit.fr](https://duckit.fr) - [Ali Hmaou](https://www.linkedin.com/in/ali-hmaou-6b7b73146/)*